<h1 align=center><font size = 5> Analyzing Subway Stations in Toronto</font></h1>

## Table of Contents

1. <a href="#item1">Background</a>    
2. <a href="#item2">Questions and Data to be Used</a> 
3. <a href="#item3">Methodology </a>

 A) <a href="#itemA"> Data </a>
    
 B) <a href="#itemB">Methods</a>
    
      (i) <a href="#itemI"> Clustering </a>

      (ii) <a href="#itemII"> Binning </a>
      
4. <a href="#item4">Results </a>

5. <a href="#item5">Discussion, Summary and Concusions </a>

6. <a href="#item5">Raw Code </a>


## 1. Background

Toronto's main public transportation system is operated by the Toronto Transit Commission (TTC). The backbone of its public transport network is the Toronto subway system, which includes three heavy-rail rapid transit lines spanning the city, including the U-shaped Line 1 and east–west Line 2 [1]. Additional two smaller lines (3 and 4) are also part of the subway network in Toronto. Note that the city (municipality) of Toronto includes, besinde the original city of Tornoto, also North York, Etobicoke and Scarbough.   

The stations are charactersitically different one from the others. In particular, the porpuse (or characterstic) of the subway station may be related to the nearby central vanues. For example, subway station near the university has different features than station in rural area. These lead us to the first stage of the analysis, which is to examine the present subway network. 

The transit score in general, and the vicinity to subway station in particular may be one of the important features in the wellbeing of the citizents of the Great Toronoto Area (GTA). Findings suggest that low-income households, on average, see greater improvement in travel time savings to destinations in Toronto than other groups and the overall population [2]        

Ideally, the subway network, together with other TTC services (e.g. buses, streetcar), is expected to cover the following areas. The departure stations are located in densed areas, and in low-income households neighborhood (where one cannot effort car for example), and destination stations are in the vecinity of buisness area, shoping mall, tourist popular venues, or university.    

Analyzing the present state of the GTA's subway network, together with spatial and demographical features of the neighboorhoods themselfs will help the leader to decide (where assume limited scources) wheather this current situation is satisfacory or that urgent new subway stations or lines must be added. 

## 2. Questions and Data to be Used

**Goal:**

The main questions surface in this analysis are: 

1) What are the main features of the subway stations? (e.g. the group might be end-stations, transfer to other lines stations, stations near uniqie featuers such as airpost, univierty, malls) 

2) How the differnt groups of stations spatilaly distributed?

3) Is there a relation beween the demographic features of the neighborhoods the subway crossing them? 

4) Can we suggest improvemnt  or additional subway stations that need to be build using the data from the above questions. 

**Data and Methods:**

*A: Clustering Subway Stations Using Nearbe Venues*

To analyzed the current state of the subway network, together wuth the differnt features of each station, I will use the FourSquares database. First, I will extarct from FourSquares the subway stations in Toronto. Next, I will explore the nearby vanues for every subway stations found in prevous step. In the last step of the examination the current state, I will cluster the stations using similarities on their nearby values.

*B: Clustering Subway Stations Using Demographical Features of the Neighborhood*

To explore the current state of the subway netwrok, in relation to the dempgraphyical features of the population nearby, I will use data piblished in Wikipedia [3], where I will concentrate of two features of each neighborhood; yearly income, and its density (i.e population/areal unit). Here, I want to eaxmine wheather the current state if satispfying. Here, I since we expect that low income households communities and dense neighboorhood, will be more benefical from having subway station nearby, we hope to find these neighboorhood transit score to be high. 

*C: Clustering Toronto Neighborhoods Using Demographical Features*
Here I assume that the current state of the subway network and the number of stations are not sufficint and some improvemnt must be done. By exploring the most dense neighboorhoods or neighborhoods with low-income household, we can suggest improvemnt to the current routes, and to detect what neighboorhoods that need (relatiavly urgently) new stations.  Since [3] includes demograhpic data by neighborhoods' names, I also using [4] to find the location of each neighborhood


### References

[1] https://en.wikipedia.org/wiki/Public_transportation_in_Toronto

[2] http://www.metrolinx.com/en/docs/pdf/board_agenda/20190912/20190912_BoardMtg_Ontario_Line_Initial_Business_Case_EN.pdf

[3] https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods

[4] https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

## 6. Raw Code

Import all libraries needed

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  openssl                                 1.1.1g-h516909a_0 --> 1.1.1g-h516909a_1



openssl-1.1.1g       | 2.1 MB    | #####

In [2]:
!pip install lxml

     |████████████████████████████████| 5.5MB 21.0MB/s eta 0:00:01


In [3]:
from lxml import html

**Section A: Clustering Subway Stations Using Nearbe Venues**

In [4]:
CLIENT_ID = '3YE122SBZW2UXKT3OT2NHF53ZCVIVAGCQUY12PKWO1JG34KA' # your Foursquare ID
CLIENT_SECRET = 'MX25MFGNMBQNNLFCW4FPI2ZXG0HDVWG2YW5FIIH3H0UIRSD1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3YE122SBZW2UXKT3OT2NHF53ZCVIVAGCQUY12PKWO1JG34KA
CLIENT_SECRET:MX25MFGNMBQNNLFCW4FPI2ZXG0HDVWG2YW5FIIH3H0UIRSD1


In [5]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Searching the subway stations locations using FourSquares 

In [6]:
search_query = 'subway station'
LIMIT=200
radius=500000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3YE122SBZW2UXKT3OT2NHF53ZCVIVAGCQUY12PKWO1JG34KA&client_secret=MX25MFGNMBQNNLFCW4FPI2ZXG0HDVWG2YW5FIIH3H0UIRSD1&ll=43.6534817,-79.3839347&v=20180604&query=subway station&radius=500000&limit=200'

In [7]:
results = requests.get(url).json()

In [8]:
venues = results['response']['venues']

# tranform venues into a dataframe
SubwayStation_location = json_normalize(venues)

SubwayStation_location.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4ae75aaaf964a520a4aa21e3,Osgoode Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597091895,False,250 University Ave.,at Queen St. W.,43.650877,-79.386824,"[{'label': 'display', 'lat': 43.65087700047692...",371,M5H 3E5,CA,Toronto,ON,Canada,"[250 University Ave. (at Queen St. W.), Toront...",NaN
1,4ad7b1cbf964a520bf0d21e3,Queen Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597091895,False,171 Yonge St,at Queen St,43.652373,-79.379191,"[{'label': 'display', 'lat': 43.65237332981179...",401,M5C 2L7,CA,Toronto,ON,Canada,"[171 Yonge St (at Queen St), Toronto ON M5C 2L...",St. Lawrence
2,4ad7b160f964a520ba0d21e3,Dundas Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597091895,False,300 Yonge St,under Dundas St,43.656096,-79.380785,"[{'label': 'display', 'lat': 43.65609649653765...",386,M5G 2B3,CA,Toronto,ON,Canada,"[300 Yonge St (under Dundas St), Toronto ON M5...",Downtown Toronto
3,4b04a80bf964a520f95522e3,St Patrick Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597091895,False,449 University Ave.,at Dundas St. W.,43.654818,-79.388331,"[{'label': 'display', 'lat': 43.65481752615579...",384,M5G 1W8,CA,Toronto,ON,Canada,"[449 University Ave. (at Dundas St. W.), Toron...",NaN
4,4ad8f3e4f964a5205c1621e3,St Andrew Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597091895,False,147 University Ave,at King St W,43.647773,-79.384939,"[{'label': 'display', 'lat': 43.64777286099943...",640,M5H 1J9,CA,Toronto,ON,Canada,"[147 University Ave (at King St W), Toronto ON...",NaN


In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'categories']
    
    return(nearby_venues)

In [10]:
SubwayStation_venues = getNearbyVenues(names=SubwayStation_location['name'],
                                   latitudes=SubwayStation_location['location.lat'],
                                   longitudes=SubwayStation_location['location.lng']) 

Osgoode Subway Station
Queen Subway Station
Dundas Subway Station
St Patrick Subway Station
St Andrew Subway Station
Queen's Park Subway Station
Union Subway Station
Wellesley Subway Station
King Subway Station
Spadina Subway Station
College Subway Station
Sherbourne Subway Station
Bloor-Yonge Subway Station
St. George Subway Station
Museum Subway Station
Bay Subway Station
St Clair West Subway Station
Broadview Subway Station
Donlands Subway Station
Rosedale Subway Station
Castle Frank Subway Station
Ossington Subway Station
Wilson Subway Station
Christie Subway Station
Finch Subway Station
Pape Subway Station
Bathurst Subway Station
Kipling Subway Station
Eglinton West Subway Station
St Clair Subway Station
Lansdowne Subway Station
Islington Subway Station
Woodbine Subway Station
Main Street Subway Station
Ossington Subway Station
Lawrence West Subway Station
Dufferin Subway Station
Royal York Subway Station
Chester Subway Station
Coxwell Subway Station
Dundas West Subway Station
Egl

In [11]:
map_subway_stations = folium.Map(location=[latitude,longitude], zoom_start=12)
# add markers to map
for lat, lng, label in zip(SubwayStation_location['location.lat'], SubwayStation_location['location.lng'], SubwayStation_location['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_subway_stations)  
    
map_subway_stations

In [12]:
SubwayStation_onehot = pd.get_dummies(SubwayStation_venues[['categories']], prefix="", prefix_sep="")
SubwayStation_onehot['Neighborhood'] = SubwayStation_venues['Neighborhood']
fixed_columns = [SubwayStation_onehot.columns[-1]] + list(SubwayStation_onehot.columns[:-1])
SubwayStation_onehot = SubwayStation_onehot[fixed_columns]
SubwayStation_grouped = SubwayStation_onehot.groupby('Neighborhood').mean().reset_index()
SubwayStation_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Chinese Restaurant,Chiropractor,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Bathurst Subway Station,0.000000,0.0,0.0,0.0,0.016667,0.0,0.000000,0.000000,0.00,0.016667,0.000000,0.0,0.0,0.0,0.033333,0.000000,0.016667,0.0,0.016667,0.0,0.0,0.016667,0.0,0.016667,0.000000,0.0,0.0,0.000000,0.0,0.0,0.016667,0.0,0.0,0.000000,0.016667,0.0,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.016667,0.016667,0.0,0.000000,0.0,0.000000,0.016

In [13]:
kclusters = 5

SubwayStation_grouped_clustering = SubwayStation_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SubwayStation_grouped_clustering)

kmeans.labels_[0:20] 

array([0, 2, 2, 2, 4, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2],
      dtype=int32)

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
SubwayStation_venues_sorted = pd.DataFrame(columns=columns)
SubwayStation_venues_sorted['Neighborhood'] = SubwayStation_grouped['Neighborhood']

for ind in np.arange(SubwayStation_grouped.shape[0]):
    SubwayStation_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SubwayStation_grouped.iloc[ind, :], num_top_venues)

SubwayStation_venues_sorted.head(2)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Subway Station,Korean Restaurant,Grocery Store,Bakery,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Eastern European Restaurant,Japanese Restaurant,Bubble Tea Shop,Burrito Place
1,Bay Subway Station,Boutique,Italian Restaurant,Coffee Shop,French Restaurant,Hotel,Restaurant,Japanese Restaurant,Spa,Café,Sushi Restaurant


In [16]:
# add clustering labels
SubwayStation_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

SubwayStation_merged = SubwayStation_location

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
SubwayStation_merged = SubwayStation_merged.join(SubwayStation_venues_sorted.set_index('Neighborhood'), on='name')


SubwayStation_merged.head(3) 

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4ae75aaaf964a520a4aa21e3,Osgoode Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597091895,False,250 University Ave.,at Queen St. W.,43.650877,-79.386824,"[{'label': 'display', 'lat': 43.65087700047692...",371,M5H 3E5,CA,Toronto,ON,Canada,"[250 University Ave. (at Queen St. W.), Toront...",NaN,2,Café,Coffee Shop,Hotel,Bar,Steakhouse,Thai Restaurant,Salad Place,Pub,Gym,Pizza Place
1,4ad7b1cbf964a520bf0d21e3,Queen Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597091895,False,171 Yonge St,at Queen St,43.652373,-79.379191,"[{'label': 'display', 'lat': 43.65237332981179...",401,M5C 2L7,CA,Toronto,ON,Canada,"[171 Yonge St (at Queen St), Toronto ON M5C 2L...",St. Lawrence,2,Coffee Shop,Clothing Store,Restaurant,American Restaurant,Gym,Cosmetics Shop,Café,Italian Restaurant,Gastropub,Seafood Restaurant
2,4ad7b160f964a520ba0d21e3,Dundas Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597091895,False,300 Yonge St,under Dundas St,43.656096,-79.380785,"[{'label': 'display', 'lat': 43.65609649653765...",386,M5G 2B3,CA,Toronto,ON,Canada,"[300 Yonge St (under Dundas St), Toronto ON M5...",Downtown Toronto,2,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Plaza,Hotel,Diner,Café,Movie Theater,Sushi Restaurant,Bubble Tea Shop


In [17]:
# create map
SubwayStation_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SubwayStation_merged['location.lat'], SubwayStation_merged['location.lng'], SubwayStation_merged['name'], SubwayStation_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(SubwayStation_map_clusters)
       
SubwayStation_map_clusters

In [18]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 0, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Christie Subway Station,15 Christie St.,at Bloor St. W.,43.664058,-79.418509,"[{'label': 'display', 'lat': 43.66405844790849...",3023,M6G 3B1,CA,Toronto,ON,Canada,"[15 Christie St. (at Bloor St. W.), Toronto ON...",NaN,0,Korean Restaurant,Coffee Shop,Grocery Store,Café,Indian Restaurant,Ice Cream Shop,Sandwich Place,Mexican Restaurant,Dessert Shop,Cocktail Bar
24,Finch Subway Station,5714 Yonge St,at Finch Ave,43.780917,-79.415113,"[{'label': 'display', 'lat': 43.78091670477462...",14405,M2M 4E7,CA,Toronto,ON,Canada,"[5714 Yonge St (at Finch Ave), Toronto ON M2M ...",NaN,0,Korean Restaurant,Bubble Tea Shop,Coffee Shop,Japanese Restaurant,Vietnamese Restaurant,Dessert Shop,Gym,Salad Place,Food Stand,Smoothie Shop
26,Bathurst Subway Station,565 Bloor St W,at Bathurst St,43.666031,-79.411317,"[{'label': 'display', 'lat': 43.66603075925603...",2610,M5S 1Y5,CA,Toronto,ON,Canada,"[565 Bloor St W (at Bathurst St), Toronto ON M...",NaN,0,Korean Restaurant,Grocery Store,Bakery,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Eastern European Restaurant,Japanese Restaurant,Bubble Tea Shop,Burrito Place


In [19]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 1, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Warden Subway Station Passenger Pickup,3276 St Clair Ave E,at Warden Ave,43.711268,-79.279295,"[{'label': 'display', 'lat': 43.71126791438669...",10599,NaN,CA,Toronto,ON,Canada,"[3276 St Clair Ave E (at Warden Ave), Toronto ...",Clairlea - Birchmount,1,Bakery,Intersection,Metro Station,Bus Line,Bus Station,Park,Women's Store,Electronics Store,Ethiopian Restaurant,Event Space
44,Warden Subway Station,3276 St Clair Ave E,at Warden Ave,43.711229,-79.279602,"[{'label': 'display', 'lat': 43.71122905571982...",10576,M1L 1W1,CA,Toronto,ON,Canada,"[3276 St Clair Ave E (at Warden Ave), Toronto ...",NaN,1,Bakery,Intersection,Metro Station,Bus Line,Bus Station,Park,Women's Store,Electronics Store,Ethiopian Restaurant,Event Space


In [20]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 2, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Osgoode Subway Station,250 University Ave.,at Queen St. W.,43.650877,-79.386824,"[{'label': 'display', 'lat': 43.65087700047692...",371,M5H 3E5,CA,Toronto,ON,Canada,"[250 University Ave. (at Queen St. W.), Toront...",NaN,2,Café,Coffee Shop,Hotel,Bar,Steakhouse,Thai Restaurant,Salad Place,Pub,Gym,Pizza Place
1,Queen Subway Station,171 Yonge St,at Queen St,43.652373,-79.379191,"[{'label': 'display', 'lat': 43.65237332981179...",401,M5C 2L7,CA,Toronto,ON,Canada,"[171 Yonge St (at Queen St), Toronto ON M5C 2L...",St. Lawrence,2,Coffee Shop,Clothing Store,Restaurant,American Restaurant,Gym,Cosmetics Shop,Café,Italian Restaurant,Gastropub,Seafood Restaurant
2,Dundas Subway Station,300 Yonge St,under Dundas St,43.656096,-79.380785,"[{'label': 'display', 'lat': 43.65609649653765...",386,M5G 2B3,CA,Toronto,ON,Canada,"[300 Yonge St (under Dundas St), Toronto ON M5...",Downtown Toronto,2,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Plaza,Hotel,Diner,Café,Movie Theater,Sushi Restaurant,Bubble Tea Shop
3,St Patrick Subway Station,449 University Ave.,at Dundas St. W.,43.654818,-79.388331,"[{'label': 'display', 'lat': 43.65481752615579...",384,M5G 1W8,CA,Toronto,ON,Canada,"[449 University Ave. (at Dundas St. W.), Toron...",NaN,2,Café,Coffee Shop,Art Gallery,Japanese Restaurant,Bubble Tea Shop,Office,Chinese Restaurant,French Restaurant,Miscellaneous Shop,Sushi Restaurant
4,St Andrew Subway Station,147 University Ave,at King St W,43.647773,-79.384939,"[{'label': 'display', 'lat': 43.64777286099943...",640,M5H 1J9,CA,Toronto,ON,Canada,"[147 University Ave (at King St W), Toronto ON...",NaN,2,Coffee Shop,Hotel,Italian Restaurant,Seafood Restaurant,American Restaurant,Café,Restaurant,Asian Restaurant,Salon / Barbershop,Theater
5,Queen's Park Subway Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",NaN,2,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bubble Tea Shop,Restaurant,Bookstore
6,Union Subway Station,55 Front St W,at Bay St,43.645602,-79.379793,"[{'label': 'display', 'lat': 43.64560188525978...",938,M5J 1E6,CA,Toronto,ON,Canada,"[55 Front St W (at Bay St), Toronto ON M5J 1E6...",NaN,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,American Restaurant,Deli / Bodega,Bakery,Gastropub,Salad Place
7,Wellesley Subway Station,551 Yonge St.,at Wellesley St.,43.665077,-79.383709,"[{'label': 'display', 'lat': 43.66507686349976...",1290,M4Y 1Z2,CA,Toronto,ON,Canada,"[551 Yonge St. (at Wellesley St.), Toronto ON ...",NaN,2,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Burger Joint,Café,Sushi Restaurant,Diner,Bubble Tea Shop
8,King Subway Station,70 Yonge St,under King St,43.649165,-79.377795,"[{'label': 'display', 'lat': 43.64916502643937...",689,M5H 1A1,CA,Toronto,ON,Canada,"[70 Yonge St (under King St), Toronto ON M5H 1...",NaN,2,Coffee Shop,Restaurant,Hotel,Café,Seafood Restaurant,Gastropub,Japanese Restaurant,Gym,American Restaurant,Beer Bar
9,Spadina Subway Station,371 Bloor St. W.,at Spadina Rd.,43.667289,-79.403943,"[{'label': 'display', 'lat': 43.66728853654419...",2226,M5S 2R7,CA,Toronto,ON,Canada,"[371 Bloor St. W. (at Spadina Rd.), Toronto ON...",NaN,2,Café,Sandwich Place,Italian Restaurant,Bookstore,Bakery,Japanese Restaurant,Pub,Coffee Shop,Eastern European Restaurant,Bank


In [21]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 3, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Victoria Park Subway Station,777 Victoria Park Ave,at Danforth,43.694883,-79.288735,"[{'label': 'display', 'lat': 43.69488264493771...",8943,M4B 2E2,CA,Toronto,ON,Canada,"[777 Victoria Park Ave (at Danforth), Toronto ...",NaN,3,Gas Station,Metro Station,Bank,Spa,Restaurant,Liquor Store,Beer Store,Golf Course,Electronics Store,Donut Shop


In [22]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 4, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Castle Frank Subway Station,600 Bloor St E,at Castle Frank Rd.,43.673785,-79.368647,"[{'label': 'display', 'lat': 43.67378501123131...",2573,M4W 3Z2,CA,Toronto,ON,Canada,"[600 Bloor St E (at Castle Frank Rd.), Toronto...",NaN,4,Convenience Store,Park,Nightclub,Outdoors & Recreation,Metro Station,Deli / Bodega,Dance Studio,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


**Section B: Clustering Subway Stations Using Dempgraphic Features**

In [23]:
 data_demo=pd.read_html(r'https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods')

In [24]:
demo_features=data_demo[5]
demo_features.head(2)

,Name,FM,Census Tracts,Population,Land area (km2),Density (people/km2),% Change in Population since 2001,Average Income,Transit Commuting %,% Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map
0,Toronto CMA Average,NaN,All,5113149.0,5903.63,866.0,9.0,40704.0,10.6,11.4,NaN,NaN,NaN
1,Agincourt,S,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...",44577.0,12.45,3580.0,4.6,25750.0,11.1,5.9,Cantonese (19.3%),19.3% Cantonese,NaN


In [25]:
demo_features.drop(['FM','Census Tracts','Land area (km2)','Second most common language (after English) by name','% Change in Population since 2001','% Renters','Second most common language (after English) by percentage','Map'], axis=1, inplace=True)

In [26]:
demo_features.head()

,Name,Population,Density (people/km2),Average Income,Transit Commuting %
0,Toronto CMA Average,5113149.0,866.0,40704.0,10.6
1,Agincourt,44577.0,3580.0,25750.0,11.1
2,Alexandra Park,4355.0,13609.0,19687.0,13.8
3,Allenby,2513.0,4333.0,245592.0,5.2
4,Amesbury,17318.0,4934.0,27546.0,16.4


In [27]:
df=pd.read_html(r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df_toronto=df[0]
df_toronto=df_toronto[df_toronto.Borough!='Not assigned']
df_toronto = df_toronto.reset_index(drop=True)
df_toronto.Neighbourhood = np.where(df_toronto.Neighbourhood != 'Not assigned',df_toronto.Neighbourhood,df_toronto.Borough)
df1=pd.read_csv(r"http://cocl.us/Geospatial_data")
df_new = pd.merge(df_toronto,
                 df1[['Postal Code', 'Latitude', 'Longitude']],
                 on='Postal Code', 
                 how='left')
df_new.rename(columns={'Neighbourhood':'Name','Postal Code':'zip'}, inplace=True)
df_new.head(3)

,zip,Borough,Name,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636


In [28]:
a = pd.DataFrame(df_new.Name.str.split(',').tolist(),index=df_new.zip).stack()
a = a.reset_index([0,'zip'])
a.rename(columns={0:'Name'}, inplace=True)
df1.rename(columns={'Postal Code': 'zip'},inplace=True)
dfa = pd.merge(a,
                 df1[['zip', 'Latitude', 'Longitude']],
                 on='zip', 
                 how='left')
dfa.tail(5)

,zip,Name,Latitude,Longitude
212,M8Z,Mimico NW,43.628841,-79.520999
213,M8Z,The Queensway West,43.628841,-79.520999
214,M8Z,South of Bloor,43.628841,-79.520999
215,M8Z,Kingsway Park South West,43.628841,-79.520999
216,M8Z,Royal York South West,43.628841,-79.520999


In [29]:
dem = pd.merge(demo_features,
                 dfa[['Latitude', 'Longitude','Name']],
                 on='Name', 
                 how='left')
dem.shape

(164, 7)

In [30]:
dem.tail(10)

,Name,Population,Density (people/km2),Average Income,Transit Commuting %,Latitude,Longitude
154,Wexford,17844.0,2239.0,28556.0,14.0,43.750072,-79.295849
155,Willowdale,43144.0,5618.0,39895.0,15.6,43.789053,-79.408493
156,Willowdale,43144.0,5618.0,39895.0,15.6,43.770120,-79.408493
157,Willowdale,43144.0,5618.0,39895.0,15.6,43.782736,-79.442259
158,Wilson Heights,13732.0,3317.0,37978.0,15.9,NaN,NaN
159,Woburn,48507.0,3636.0,26190.0,13.3,43.770992,-79.216917
160,Wychwood,4182.0,6150.0,53613.0,17.1,NaN,NaN
161,York Mills,17564.0,2409.0,92099.0,10.0,43.757490,-79.374714
162,York University Heights,26140.0,1979.0,24432.0,15.2,NaN,NaN
163,Yorkville,6045.0,10795.0,105239.0,18.3,NaN,NaN


In [31]:
dem.dropna(inplace=True)
dem.shape

(49, 7)

In [32]:
dem.head()

,Name,Population,Density (people/km2),Average Income,Transit Commuting %,Latitude,Longitude
1,Agincourt,44577.0,3580.0,25750.0,11.1,43.794200,-79.262029
7,Bathurst Manor,14945.0,3187.0,34169.0,13.4,43.754328,-79.442259
9,Bayview Village,12280.0,2966.0,46752.0,14.4,43.786947,-79.385975
11,Bedford Park,13749.0,6057.0,80827.0,15.2,43.733283,-79.419750
13,Birch Cliff,12266.0,3525.0,48965.0,11.4,43.692657,-79.264848


In [33]:
map_toronto_neighborhoods = folium.Map(location=[latitude,longitude], zoom_start=12)
# add markers to map
for lat, lng, label in zip(dem['Latitude'], dem['Longitude'], dem['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighborhoods)  
    
map_toronto_neighborhoods

In [34]:
kclusters = 10

dem.columns[4]
Neighborhoods_income = dem.drop(['Name','Population','Density (people/km2)','Transit Commuting\xa0%','Latitude','Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Neighborhoods_income)



In [35]:
# add clustering labels
#Neighborhoods_income.insert(0, 'Cluster Labels', kmeans.labels_)

Neighborhoods_income.head()

,Average Income
1,25750.0
7,34169.0
9,46752.0
11,80827.0
13,48965.0


In [36]:
Neighborhoods_income_merged = Neighborhoods_income

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Neighborhoods_income_merged = pd.merge(Neighborhoods_income,
                 dem[['Name', 'Latitude', 'Longitude']],
                 left_index=True, right_index=True)
#Neighborhoods_income_merged = Neighborhoods_income_merged.join(dem)

Neighborhoods_income_merged.head(3) 

,Average Income,Name,Latitude,Longitude
1,25750.0,Agincourt,43.794200,-79.262029
7,34169.0,Bathurst Manor,43.754328,-79.442259
9,46752.0,Bayview Village,43.786947,-79.385975


In [37]:
print(Neighborhoods_income_merged['Average Income'].max())
print(Neighborhoods_income_merged['Average Income'].min())

214110.0
22341.0


In [38]:
# create map
Neighborhoods_income_clusters_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Neighborhoods_income_merged['Latitude'], Neighborhoods_income_merged['Longitude'], Neighborhoods_income_merged['Name'],Neighborhoods_income_merged['Average Income']):
    label = folium.Popup(str(poi) + ' income ' + str(cluster), parse_html=True)
    clust=int(cluster/((214110.0-22341.0)/10))-1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clust-1],
        fill=True,
        fill_color=rainbow[clust-1],
        fill_opacity=0.7).add_to(Neighborhoods_income_clusters_map)
       
Neighborhoods_income_clusters_map

In [39]:
for lat, lng, label in zip(SubwayStation_location['location.lat'], SubwayStation_location['location.lng'], SubwayStation_location['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(Neighborhoods_income_clusters_map)

In [40]:
Neighborhoods_income_clusters_map

In [41]:
print(dem['Population'].max())
print(dem['Population'].min())

48507.0
3740.0


In [42]:
# create map
Neighborhoods_popultaion_clusters_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dem['Latitude'], dem['Longitude'], dem['Name'],dem['Population']):
    label = folium.Popup(str(poi) + ' population ' + str(cluster), parse_html=True)
    clust=int(cluster/((48507.0-3740.0)/10))-1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clust-1],
        fill=True,
        fill_color=rainbow[clust-1],
        fill_opacity=0.7).add_to(Neighborhoods_popultaion_clusters_map)
       
for lat, lng, label in zip(SubwayStation_location['location.lat'], SubwayStation_location['location.lng'], SubwayStation_location['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(Neighborhoods_popultaion_clusters_map)
        
Neighborhoods_popultaion_clusters_map

In [43]:
print(dem['Density (people/km2)'].max())
print(dem['Density (people/km2)'].min())

63765.0
1618.0


In [59]:
# create map
Neighborhoods_popultaion_density_clusters_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dem['Latitude'], dem['Longitude'], dem['Name'],dem['Population']):
    label = folium.Popup(str(poi) + ' population denity ' + str(cluster), parse_html=True)
    clust=int(cluster/((63765.0-1618.0)/10))-1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clust-1],
        fill=True,
        fill_color=rainbow[clust-1],
        fill_opacity=0.7).add_to(Neighborhoods_popultaion_density_clusters_map)
       
for lat, lng, label in zip(SubwayStation_location['location.lat'], SubwayStation_location['location.lng'], SubwayStation_location['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.8,
        parse_html=False).add_to(Neighborhoods_popultaion_density_clusters_map)
        
Neighborhoods_popultaion_density_clusters_map

## 4. Methodology

### 4A) Data 

- Finding the subway station location using FourSquere 

we use only the relevant data for us; the name of the subway station, and some location data (adress, postal code, latitiude and longitide)   

In [45]:
SubwayStation_location.drop(['id','categories','referralId','hasPerk','location.labeledLatLngs','location.distance','location.cc','location.city','location.state','location.country','location.formattedAddress','location.neighborhood'],1,inplace=True)
SubwayStation_location.head()

,name,location.address,location.crossStreet,location.lat,location.lng,location.postalCode
0,Osgoode Subway Station,250 University Ave.,at Queen St. W.,43.650877,-79.386824,M5H 3E5
1,Queen Subway Station,171 Yonge St,at Queen St,43.652373,-79.379191,M5C 2L7
2,Dundas Subway Station,300 Yonge St,under Dundas St,43.656096,-79.380785,M5G 2B3
3,St Patrick Subway Station,449 University Ave.,at Dundas St. W.,43.654818,-79.388331,M5G 1W8
4,St Andrew Subway Station,147 University Ave,at King St W,43.647773,-79.384939,M5H 1J9


- Next to each subway station we use FourSquare to find its nearby venues

In [46]:
SubwayStation_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Bathurst Subway Station,Korean Restaurant,Grocery Store,Bakery,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Eastern European Restaurant,Japanese Restaurant,Bubble Tea Shop,Burrito Place
1,2,Bay Subway Station,Boutique,Italian Restaurant,Coffee Shop,French Restaurant,Hotel,Restaurant,Japanese Restaurant,Spa,Café,Sushi Restaurant
2,2,Bloor-Yonge Subway Station,Coffee Shop,Italian Restaurant,Café,Spa,Sushi Restaurant,Hotel,Japanese Restaurant,Yoga Studio,Boutique,Restaurant
3,2,Broadview Subway Station,Pub,Coffee Shop,Burger Joint,Restaurant,Pizza Place,Convenience Store,Sandwich Place,Café,Rental Car Location,Ramen Restaurant
4,4,Castle Frank Subway Station,Convenience Store,Park,Nightclub,Outdoors & Recreation,Metro Station,Deli / Bodega,Dance Studio,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


- To characterize the neighborhoods of Tornroto, I use the data from Wikipedia [3]. This gives us the population, density, Average income and Transit Cummuting percetage for each neighborhood

In [47]:
demo_features.head()

,Name,Population,Density (people/km2),Average Income,Transit Commuting %
0,Toronto CMA Average,5113149.0,866.0,40704.0,10.6
1,Agincourt,44577.0,3580.0,25750.0,11.1
2,Alexandra Park,4355.0,13609.0,19687.0,13.8
3,Allenby,2513.0,4333.0,245592.0,5.2
4,Amesbury,17318.0,4934.0,27546.0,16.4


- Since the dempgraphics of neighborhoods do not include their location, I merge the above DataFrame with the data from [4], thus my new table will include the locations

In [48]:
dfa.head()

,zip,Name,Latitude,Longitude
0,M3A,Parkwoods,43.753259,-79.329656
1,M4A,Victoria Village,43.725882,-79.315572
2,M5A,Regent Park,43.654260,-79.360636
3,M5A,Harbourfront,43.654260,-79.360636
4,M6A,Lawrence Manor,43.718518,-79.464763


(comment: some zip codes are not associated with neighborhood (e.g. CN tower has its own postal code), thus the upper tables contains more rows than the dempgrphic table. I droped these rows)

- merging the location data with the dempgraphic data gives the following data table

In [49]:
dem.head()

,Name,Population,Density (people/km2),Average Income,Transit Commuting %,Latitude,Longitude
1,Agincourt,44577.0,3580.0,25750.0,11.1,43.794200,-79.262029
7,Bathurst Manor,14945.0,3187.0,34169.0,13.4,43.754328,-79.442259
9,Bayview Village,12280.0,2966.0,46752.0,14.4,43.786947,-79.385975
11,Bedford Park,13749.0,6057.0,80827.0,15.2,43.733283,-79.419750
13,Birch Cliff,12266.0,3525.0,48965.0,11.4,43.692657,-79.264848


### 4B) Methods

**(i) Clustering**

- We use k-clustering (K=5) to characterize the subway station in Toronto

**(ii) Binning**

- I binned the average outcome for a given neighborhood for 10 equal sized bins. Then the data is presented with color scale between red (low average income) and blue (high aberage income)  

**(iii) Geographical Mapping**

- Using Folium maps to alalyzing the geographical pattern 

## 5) Results

The subway stations map: 

In [50]:
map_subway_stations

We are trying to see wheather there are some pattern on the subway stations using the nearby venues. We cluster the stations into 5 clusters. The results for the 5 different clusers are given the following map    

In [51]:
SubwayStation_map_clusters

In [52]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 0, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Christie Subway Station,15 Christie St.,at Bloor St. W.,43.664058,-79.418509,"[{'label': 'display', 'lat': 43.66405844790849...",3023,M6G 3B1,CA,Toronto,ON,Canada,"[15 Christie St. (at Bloor St. W.), Toronto ON...",NaN,0,Korean Restaurant,Coffee Shop,Grocery Store,Café,Indian Restaurant,Ice Cream Shop,Sandwich Place,Mexican Restaurant,Dessert Shop,Cocktail Bar
24,Finch Subway Station,5714 Yonge St,at Finch Ave,43.780917,-79.415113,"[{'label': 'display', 'lat': 43.78091670477462...",14405,M2M 4E7,CA,Toronto,ON,Canada,"[5714 Yonge St (at Finch Ave), Toronto ON M2M ...",NaN,0,Korean Restaurant,Bubble Tea Shop,Coffee Shop,Japanese Restaurant,Vietnamese Restaurant,Dessert Shop,Gym,Salad Place,Food Stand,Smoothie Shop
26,Bathurst Subway Station,565 Bloor St W,at Bathurst St,43.666031,-79.411317,"[{'label': 'display', 'lat': 43.66603075925603...",2610,M5S 1Y5,CA,Toronto,ON,Canada,"[565 Bloor St W (at Bathurst St), Toronto ON M...",NaN,0,Korean Restaurant,Grocery Store,Bakery,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Eastern European Restaurant,Japanese Restaurant,Bubble Tea Shop,Burrito Place


In [53]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 1, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Warden Subway Station Passenger Pickup,3276 St Clair Ave E,at Warden Ave,43.711268,-79.279295,"[{'label': 'display', 'lat': 43.71126791438669...",10599,NaN,CA,Toronto,ON,Canada,"[3276 St Clair Ave E (at Warden Ave), Toronto ...",Clairlea - Birchmount,1,Bakery,Intersection,Metro Station,Bus Line,Bus Station,Park,Women's Store,Electronics Store,Ethiopian Restaurant,Event Space
44,Warden Subway Station,3276 St Clair Ave E,at Warden Ave,43.711229,-79.279602,"[{'label': 'display', 'lat': 43.71122905571982...",10576,M1L 1W1,CA,Toronto,ON,Canada,"[3276 St Clair Ave E (at Warden Ave), Toronto ...",NaN,1,Bakery,Intersection,Metro Station,Bus Line,Bus Station,Park,Women's Store,Electronics Store,Ethiopian Restaurant,Event Space


In [54]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 2, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Osgoode Subway Station,250 University Ave.,at Queen St. W.,43.650877,-79.386824,"[{'label': 'display', 'lat': 43.65087700047692...",371,M5H 3E5,CA,Toronto,ON,Canada,"[250 University Ave. (at Queen St. W.), Toront...",NaN,2,Café,Coffee Shop,Hotel,Bar,Steakhouse,Thai Restaurant,Salad Place,Pub,Gym,Pizza Place
1,Queen Subway Station,171 Yonge St,at Queen St,43.652373,-79.379191,"[{'label': 'display', 'lat': 43.65237332981179...",401,M5C 2L7,CA,Toronto,ON,Canada,"[171 Yonge St (at Queen St), Toronto ON M5C 2L...",St. Lawrence,2,Coffee Shop,Clothing Store,Restaurant,American Restaurant,Gym,Cosmetics Shop,Café,Italian Restaurant,Gastropub,Seafood Restaurant
2,Dundas Subway Station,300 Yonge St,under Dundas St,43.656096,-79.380785,"[{'label': 'display', 'lat': 43.65609649653765...",386,M5G 2B3,CA,Toronto,ON,Canada,"[300 Yonge St (under Dundas St), Toronto ON M5...",Downtown Toronto,2,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Plaza,Hotel,Diner,Café,Movie Theater,Sushi Restaurant,Bubble Tea Shop
3,St Patrick Subway Station,449 University Ave.,at Dundas St. W.,43.654818,-79.388331,"[{'label': 'display', 'lat': 43.65481752615579...",384,M5G 1W8,CA,Toronto,ON,Canada,"[449 University Ave. (at Dundas St. W.), Toron...",NaN,2,Café,Coffee Shop,Art Gallery,Japanese Restaurant,Bubble Tea Shop,Office,Chinese Restaurant,French Restaurant,Miscellaneous Shop,Sushi Restaurant
4,St Andrew Subway Station,147 University Ave,at King St W,43.647773,-79.384939,"[{'label': 'display', 'lat': 43.64777286099943...",640,M5H 1J9,CA,Toronto,ON,Canada,"[147 University Ave (at King St W), Toronto ON...",NaN,2,Coffee Shop,Hotel,Italian Restaurant,Seafood Restaurant,American Restaurant,Café,Restaurant,Asian Restaurant,Salon / Barbershop,Theater
5,Queen's Park Subway Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",NaN,2,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bubble Tea Shop,Restaurant,Bookstore
6,Union Subway Station,55 Front St W,at Bay St,43.645602,-79.379793,"[{'label': 'display', 'lat': 43.64560188525978...",938,M5J 1E6,CA,Toronto,ON,Canada,"[55 Front St W (at Bay St), Toronto ON M5J 1E6...",NaN,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,American Restaurant,Deli / Bodega,Bakery,Gastropub,Salad Place
7,Wellesley Subway Station,551 Yonge St.,at Wellesley St.,43.665077,-79.383709,"[{'label': 'display', 'lat': 43.66507686349976...",1290,M4Y 1Z2,CA,Toronto,ON,Canada,"[551 Yonge St. (at Wellesley St.), Toronto ON ...",NaN,2,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Burger Joint,Café,Sushi Restaurant,Diner,Bubble Tea Shop
8,King Subway Station,70 Yonge St,under King St,43.649165,-79.377795,"[{'label': 'display', 'lat': 43.64916502643937...",689,M5H 1A1,CA,Toronto,ON,Canada,"[70 Yonge St (under King St), Toronto ON M5H 1...",NaN,2,Coffee Shop,Restaurant,Hotel,Café,Seafood Restaurant,Gastropub,Japanese Restaurant,Gym,American Restaurant,Beer Bar
9,Spadina Subway Station,371 Bloor St. W.,at Spadina Rd.,43.667289,-79.403943,"[{'label': 'display', 'lat': 43.66728853654419...",2226,M5S 2R7,CA,Toronto,ON,Canada,"[371 Bloor St. W. (at Spadina Rd.), Toronto ON...",NaN,2,Café,Sandwich Place,Italian Restaurant,Bookstore,Bakery,Japanese Restaurant,Pub,Coffee Shop,Eastern European Restaurant,Bank


In [55]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 3, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Victoria Park Subway Station,777 Victoria Park Ave,at Danforth,43.694883,-79.288735,"[{'label': 'display', 'lat': 43.69488264493771...",8943,M4B 2E2,CA,Toronto,ON,Canada,"[777 Victoria Park Ave (at Danforth), Toronto ...",NaN,3,Gas Station,Metro Station,Bank,Spa,Restaurant,Liquor Store,Beer Store,Golf Course,Electronics Store,Donut Shop


In [56]:
SubwayStation_merged.loc[SubwayStation_merged['Cluster Labels'] == 4, SubwayStation_merged.columns[[1] + list(range(5, SubwayStation_merged.shape[1]))]]

,name,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Castle Frank Subway Station,600 Bloor St E,at Castle Frank Rd.,43.673785,-79.368647,"[{'label': 'display', 'lat': 43.67378501123131...",2573,M4W 3Z2,CA,Toronto,ON,Canada,"[600 Bloor St E (at Castle Frank Rd.), Toronto...",NaN,4,Convenience Store,Park,Nightclub,Outdoors & Recreation,Metro Station,Deli / Bodega,Dance Studio,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


After clustering the subway stations using nearby venues, we conclude that most stations share similar neearby common venues.

In the largerst cluster (marked with blue), the most common venue near the subway stations is mostly cafe or coffee shops (where in some stations it is the 2nd most common). Then, in the puprle cluster, which contains only one subway station, the most common venue is a bakery.  In the red cluster the most common venue is a Korean restaurant, which suggest a relative highly percentage of koreans in these areas.  The other two clusters also contains one subway station each. The common venue in the green cluster is a gas station and the second most common is a park, and in the orange cluster we obtain that convenience store is the most common venue and the second most common is a bank.

The suggested conclusions from the above results are given in the followng. First, the most common venue near the subway stations is cafe (or coffee shops). Second, the red, purple, green and orange stations (see map above) have unique features. The areas near red subway stations may populated with larger percentage of Koreans (compare to other stations). Furthermore, the green (Victoria Park Subway Station), and purple (Warden Subway Station) are not seem to be located in highly popultated areas, and might be that are used for transfer through the stations. The last statemnet relies on the fact that in the most common venues list there is a bus station, gas station, intersections etc. Last, In the orange cluster (Castle Frank Subway Station), the 3 most common venues are convenience store, park, and a night club, thus this station has some unique features. 

In [57]:
Neighborhoods_income_clusters_map

The above map shows us that downtown area and the north-east and north-west of the city has lower averge income. (remind that the scale from red to blue represent low to high average income (repectivly). The subway stations are given with small green dots. 

Clearly, two most-needed regions (north-east and north-west neighborhoods) do not have access to subway service. In addition, the current subway lines cross rear mid- and high-income neighborhhods. 

To clarify, I note that there might be some (temporal) correlation between the lower-income neighborhoods to lack of access to subway lines. This correlation might be accomulated during the years, e.g. a new subway line makes house prices to increas such that poorer population will not be able to continue live in this area. I do not examine such temporal effcet.  

In [58]:
Neighborhoods_popultaion_density_clusters_map

In the above map I present high (red) and low (blue) population density neighborhoods. It shows that most densed neighborhoods have access to subway. However,  some high denisity neighborhoods, including for example Humberlea or Rouge Hill,  have no (reasonable) access to subway stations. 

## 5. Discussion, Summary and Conclusions

I have shown the geographical locations of the subway stations in Toronto, and found some stations with uniques features. In addition, I have found that low-income regions, north-east and north-west, has no eccess to subway station even though their in much need. This is, in contrast, with comparision to many neighboorhoods next to the subway, with higher income on average. Thus, I suggest to connect these areas to the subway network. Moreover, I have found some neighborhoods with high density population,  where connecting these neighboorhood to the subway nework will be highly beneficial.  